<a href="https://colab.research.google.com/github/Carlinhos2023/PROJETO_IMERS-O_IA_ALURA/blob/main/Chat_bot_Empresa_First_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
#INSTALAÇÃO
!pip install -U -q google-generativeai

ERROR: Operation cancelled by user


In [2]:
# IMPORTAÇÃO DAS BIBLIOTECAS NECESSARIAS.

import google.generativeai as genai
import pandas as pd
import numpy as np

In [70]:
#API KEY
from google.colab import userdata

API_KEY=userdata.get('SECRET_KEY')
genai.configure(api_key=API_KEY)

In [6]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [8]:
title = 'ola'
sample_text = 'tudo bem'

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.023327215, -0.011880942, 0.010564977, 0.024848772, 0.023952693, 0.026523305, -0.01017627, 0.009270761, -0.014446339, -0.005868929, 0.0049321167, 0.020085251, -0.0100921225, -0.016770286, 0.020776995, -5.1781324e-05, 0.0031569214, 0.034490526, -0.0071905977, -0.010118244, 0.0197759, 0.0025616935, -0.06938286, -0.008869576, 0.037929785, -0.015200878, -0.03746988, -0.03921688, 0.009362235, -0.0072863502, -0.031348836, 0.050618492, -0.015084125, 0.026359325, 0.0034098432, -0.062679686, -0.0019191981, 0.021017881, 0.028600285, 0.008815611, 0.02352044, -0.022980282, 0.0014269042, -0.00860996, 0.027678987, -0.002939403, -0.037376117, 0.018301725, -0.042577513, -0.03400035, 0.044709932, 0.0036779165, 0.09350434, -0.0059829727, -0.015203417, -0.0016202168, 0.03664331, -0.0010415288, -0.040299915, 0.038499203, -0.0052852277, 0.021513037, 0.0721439, 0.008834239, -0.016125495, -0.047088858, -0.02074562, 0.016337449, 0.013650721, 0.038801976, -0.019781973, -0.07387683, 0.029833958,

In [60]:
#LISTAGEM DE DOCUMENTOS QUE SERÃO BUSCADOS
DOCUMENT1 = {
    "Título": "Setor de Financeiro",
    "Conteúdo": "Tania (13) 9824-2451"
}

DOCUMENT2 = {
    "Título": "Setor Comercial",
    "Conteúdo": "Fabri (13) 9824-2452"
}

DOCUMENT3 = {
    "Título": "Setor Recursos Humanos",
    "Conteúdo": "Kelly (13) 9824-2453"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [61]:
df = pd.DataFrame(documents)
df

,Título,Conteúdo
0,Setor de Financeiro,Tania (13) 9824-2451
1,Setor Comercial,Fabri (13) 9824-2452
2,Setor Recursos Humanos,Kelly (13) 9824-2453


In [62]:
model = "models/embedding-001"

In [63]:
def embed_fn(title, text):
  return genai.embed_content(model="models/embedding-001",
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [64]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Setor de Financeiro,Tania (13) 9824-2451,"[0.046324156, -0.008411146, -0.0050191716, -0...."
1,Setor Comercial,Fabri (13) 9824-2452,"[0.050148956, -0.02818091, -0.026038283, -0.02..."
2,Setor Recursos Humanos,Kelly (13) 9824-2453,"[0.058541548, -0.0034384567, -0.047014035, -0...."


In [65]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]


In [67]:
consulta = "telefone do RH?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Kelly (13) 9824-2453
